<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/2.3-merge_join-datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Este cuaderno se basa en el cuaderno de Python Data Science Handbook de Jake VanderPlas
*El texto se publica bajo la licencia CC-BY-NC-ND, y el código se publica bajo la licencia MIT.

# Combinando conjuntos de datos: Combinar y unir

In [44]:
import pandas as pd
import numpy as np

### One-to-one joins

In [45]:
df1 = pd.DataFrame({'Empleado': ['Juan', 'Jose', 'Luis', 'Santiago'],
                    'grupo': ['Contabilidad', 'Ingeniería', 'Ingeniería', 'Recursos Humanos']})
df2 = pd.DataFrame({'Empleado': ['Luis', 'Juan', 'Jose', 'Santiago'],
                    'anio_ingreso': [2004, 2008, 2012, 2014]})

print("df1 \n",df1)

print(" ")

print("df2 \n",df2)


df1 
    Empleado             grupo
0      Juan      Contabilidad
1      Jose        Ingeniería
2      Luis        Ingeniería
3  Santiago  Recursos Humanos
 
df2 
    Empleado  anio_ingreso
0      Luis          2004
1      Juan          2008
2      Jose          2012
3  Santiago          2014


Unir un solo dataframe

In [46]:
df3 = pd.merge(df1, df2)
df3

,Empleado,grupo,anio_ingreso
0,Juan,Contabilidad,2008
1,Jose,Ingeniería,2012
2,Luis,Ingeniería,2004
3,Santiago,Recursos Humanos,2014


# Many-to-one joins

Se preservan los duplicados


In [47]:
df4 = pd.DataFrame({'grupo': ['Contabilidad', 'Ingeniería', 'Recursos humanos'],
                    'Supervisor': ['Carlos', 'Guido', 'Esteban']})
pd.merge(df3, df4)

,Empleado,grupo,anio_ingreso,Supervisor
0,Juan,Contabilidad,2008,Carlos
1,Jose,Ingeniería,2012,Guido
2,Luis,Ingeniería,2004,Guido


Se repite la información del supervisor

### Many-to-many joins

Las uniones de muchos a muchos son un concepto un poco confuso conceptualmente, pero están bien definidas.
Si la columna clave tanto en el array izquierdo como en el derecho contiene duplicados, entonces el resultado es una unión de muchos a muchos.
Esto será quizás más claro con un ejemplo concreto.
Consideremos lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.
Al realizar una unión de muchos a muchos, podemos recuperar las habilidades asociadas con cualquier persona individual:


In [48]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """
    
{0}{1}
    """
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)



df5 = pd.DataFrame({'grupo': ['Contabilidad', 'Contabilidad',
                              'Ingeniería', 'Ingeniería', 'Recursos Humanos', 'Recursos Humanos'],
                    'habilidades': ['matemática', 'excel', 'codificación', 'linux',
                               'excel', 'organización']})
display('df1', 'df5', "pd.merge(df1, df5)")

,Empleado,grupo
0,Juan,Contabilidad
1,Jose,Ingeniería
2,Luis,Ingeniería
3,Santiago,Recursos Humanos
,grupo,habilidades
0,Contabilidad,matemática
1,Contabilidad,excel
2,Ingeniería,codificación
3,Ingeniería,linux
4,Recursos Humanos,excel


## Specification of the Merge Key

### The ``on`` keyword
``on`` usa el nombre de la columna

In [49]:
display('df1', 'df2', "pd.merge(df1, df2, on='Empleado')")

df1
   Empleado             grupo
0      Juan      Contabilidad
1      Jose        Ingeniería
2      Luis        Ingeniería
3  Santiago  Recursos Humanos

df2
   Empleado  anio_ingreso
0      Luis          2004
1      Juan          2008
2      Jose          2012
3  Santiago          2014

pd.merge(df1, df2, on='Empleado')
   Empleado             grupo  anio_ingreso
0      Juan      Contabilidad          2008
1      Jose        Ingeniería          2012
2      Luis        Ingeniería          2004
3  Santiago  Recursos Humanos          2014

Esto funciona si ambas tienen el mismo nombre

### The ``left_on`` and ``right_on`` keywords

Cuando las columnas no se llaman igual

In [50]:
df3 = pd.DataFrame({'nombre': ['Juan', 'Jose', 'Luis', 'Santiago'],
                    'Salario': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="Empleado", right_on="nombre")')

df1
   Empleado             grupo
0      Juan      Contabilidad
1      Jose        Ingeniería
2      Luis        Ingeniería
3  Santiago  Recursos Humanos

df3
     nombre  Salario
0      Juan    70000
1      Jose    80000
2      Luis   120000
3  Santiago    90000

pd.merge(df1, df3, left_on="Empleado", right_on="nombre")
   Empleado             grupo    nombre  Salario
0      Juan      Contabilidad      Juan    70000
1      Jose        Ingeniería      Jose    80000
2      Luis        Ingeniería      Luis   120000
3  Santiago  Recursos Humanos  Santiago    90000

Se puede borrar la columna extra

In [51]:
pd.merge(df1, df3, left_on="Empleado", right_on="nombre").drop('nombre', axis=1)

,Empleado,grupo,Salario
0,Juan,Contabilidad,70000
1,Jose,Ingeniería,80000
2,Luis,Ingeniería,120000
3,Santiago,Recursos Humanos,90000


### The ``left_index`` and ``right_index`` keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index.
For example, your data might look like this:

In [52]:
df1a = df1.set_index('Empleado')
df2a = df2.set_index('Empleado')
display('df1a', 'df2a')

,grupo
Empleado,
Juan,Contabilidad
Jose,Ingeniería
Luis,Ingeniería
Santiago,Recursos Humanos
,anio_ingreso
Empleado,
Luis,2004
Juan,2008
Jose,2012


 ``left_index`` y ``right_index`` en  ``pd.merge()``:

In [53]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,grupo
Empleado,
Juan,Contabilidad
Jose,Ingeniería
Luis,Ingeniería
Santiago,Recursos Humanos
,anio_ingreso
Empleado,
Luis,2004
Juan,2008
Jose,2012


 ``DataFrame`` implementa el ``join()`` que hace un  ``merge`` con los indices:

In [54]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,grupo
Empleado,
Juan,Contabilidad
Jose,Ingeniería
Luis,Ingeniería
Santiago,Recursos Humanos
,anio_ingreso
Empleado,
Luis,2004
Juan,2008
Jose,2012


Se pueden combinar indices y columnas

In [55]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='nombre')")

df1a
                     grupo
Empleado                  
Juan          Contabilidad
Jose            Ingeniería
Luis            Ingeniería
Santiago  Recursos Humanos

df3
     nombre  Salario
0      Juan    70000
1      Jose    80000
2      Luis   120000
3  Santiago    90000

pd.merge(df1a, df3, left_index=True, right_on='nombre')
              grupo    nombre  Salario
0      Contabilidad      Juan    70000
1        Ingeniería      Jose    80000
2        Ingeniería      Luis   120000
3  Recursos Humanos  Santiago    90000

El detalle del merge se puede ver en la documentación de panda (http://pandas.pydata.org/pandas-docs/stable/merging.html) .

In [56]:
df6 = pd.DataFrame({'nombre': ['Pedro', 'Pablo', 'Maria'],
                    'comida': ['pescado', 'frijo', 'pan']},
                   columns=['nombre', 'comida'])
df7 = pd.DataFrame({'nombre': ['Maria', 'Jose'],
                    'bebida': ['vino', 'cerveza']},
                   columns=['nombre', 'bebida'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7)
  nombre comida bebida
0  Maria    pan   vino

Sólo tienen en común a María

In [57]:
pd.merge(df6, df7, how='inner')

,nombre,comida,bebida
0,Maria,pan,vino


Otras opciones para el ``how`` = ``'outer'``, ``'left'``, y ``'right'``. Outer toma todos los datos y llena con nulos los datos vacios

In [38]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7, how='outer')
  nombre   comida   bebida
0  Pedro  pescado      NaN
1  Pablo    frijo      NaN
2  Maria      pan     vino
3   Jose      NaN  cerveza

 *left join* y  *right join* 

In [58]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7, how='left')
  nombre   comida bebida
0  Pedro  pescado    NaN
1  Pablo    frijo    NaN
2  Maria      pan   vino

In [59]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

df6
  nombre   comida
0  Pedro  pescado
1  Pablo    frijo
2  Maria      pan

df7
  nombre   bebida
0  Maria     vino
1   Jose  cerveza

pd.merge(df6, df7, how='right')
  nombre comida   bebida
0  Maria    pan     vino
1   Jose    NaN  cerveza

Conflicto en el nombre de las columnas


In [60]:
df8 = pd.DataFrame({'nombre': ['Alberto', 'Juan', 'Luis', 'Santiago'],
                    'ranking': [1, 2, 3, 4]})
df9 = pd.DataFrame({'nombre': ['Alberto', 'Juan', 'Luis', 'Santiago'],
                    'ranking': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on=["nombre", "ranking"])')

,nombre,ranking
0,Alberto,1
1,Juan,2
2,Luis,3
3,Santiago,4
,nombre,ranking
0,Alberto,3
1,Juan,1
2,Luis,4
3,Santiago,2
,nombre,ranking


In [61]:
display('df8', 'df9', 'pd.merge(df8, df9, on=["nombre"])')

df8
     nombre  ranking
0   Alberto        1
1      Juan        2
2      Luis        3
3  Santiago        4

df9
     nombre  ranking
0   Alberto        3
1      Juan        1
2      Luis        4
3  Santiago        2

pd.merge(df8, df9, on=["nombre"])
     nombre  ranking_x  ranking_y
0   Alberto          1          3
1      Juan          2          1
2      Luis          3          4
3  Santiago          4          2

In [62]:
display('df8', 'df9', 'pd.merge(df8, df9, on="nombre", suffixes=["_L", "_R"])')

df8
     nombre  ranking
0   Alberto        1
1      Juan        2
2      Luis        3
3  Santiago        4

df9
     nombre  ranking
0   Alberto        3
1      Juan        1
2      Luis        4
3  Santiago        2

pd.merge(df8, df9, on="nombre", suffixes=["_L", "_R"])
     nombre  ranking_L  ranking_R
0   Alberto          1          3
1      Juan          2          1
2      Luis          3          4
3  Santiago          4          2